In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, 
                                     GlobalAveragePooling2D, Dense, Add, Dropout)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from collections import Counter

In [2]:
# Define dataset path
dataset_path = "/kaggle/input/medical-mnist"
categories = ['AbdomenCT', 'BreastMRI', 'Hand', 'CXR', 'HeadCT', 'ChestCT']
category_labels = {cat: idx for idx, cat in enumerate(categories)}

# Load images and labels
input_set, label_set = [], []
for category in categories:
    category_path = os.path.join(dataset_path, category)
    for img_file in os.listdir(category_path):
        img_path = os.path.join(category_path, img_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (64, 64))
        input_set.append(image)
        label_set.append(category_labels[category])

In [3]:
# Check class distribution
class_counts = Counter(label_set)
print(f"Class Distribution: {class_counts}")

# Convert to numpy arrays
input_set = np.array(input_set).reshape(-1, 64, 64, 1) / 255.0  # Normalize
label_set = tf.keras.utils.to_categorical(np.array(label_set), num_classes=len(categories))

# Train-test split
split_index = int(0.8 * len(input_set))
train_set, test_set = input_set[:split_index], input_set[split_index:]
train_labels, test_labels = label_set[:split_index], label_set[split_index:]

# Shuffle data
train_set, train_labels = shuffle(train_set, train_labels, random_state=42)
test_set, test_labels = shuffle(test_set, test_labels, random_state=42)

# Split the training data into 4 equal parts for local models
split_size = len(train_set) // 4
local_datasets = [
    (train_set[i * split_size: (i + 1) * split_size], train_labels[i * split_size: (i + 1) * split_size])
    for i in range(4)
]

Class Distribution: Counter({0: 10000, 2: 10000, 3: 10000, 4: 10000, 5: 10000, 1: 8954})


In [4]:
# Data Augmentation
data_gen = ImageDataGenerator(
    rotation_range=15,  # Rotate images randomly
    width_shift_range=0.1,  # Shift width by 10%
    height_shift_range=0.1,  # Shift height by 10%
    zoom_range=0.2,  # Zoom by 20%
    horizontal_flip=True
)

In [5]:
# Define Improved ResNet-based model
def residual_block(x, filters, kernel_size=3, strides=1):
    shortcut = x
    x = Conv2D(filters, kernel_size, strides=strides, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same', kernel_initializer='he_normal')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x

def create_resnet(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = residual_block(x, 64)
    x = residual_block(x, 128, strides=2)
    x = residual_block(x, 256, strides=2)
    x = residual_block(x, 512, strides=2)
    x = residual_block(x, 1024, strides=2)  # Increased depth
    
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)  # Reduced dropout
    outputs = Dense(num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(x)

    return Model(inputs, outputs)

In [6]:
# Federated averaging function with normalization
def fedavg(*models):
    all_weights = [model.get_weights() for model in models]
    averaged_weights = [
        np.mean(np.array([weights[i] for weights in all_weights]), axis=0)
        for i in range(len(all_weights[0]))
    ]
    return averaged_weights

In [7]:
# Initialize global model
global_model = create_resnet((64, 64, 1), len(categories))
global_model.compile(
    optimizer=Adam(learning_rate=0.0003, beta_1=0.9),  # Lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
globalweights = global_model.get_weights()

# Local models
models = [create_resnet((64, 64, 1), len(categories)) for _ in range(4)]
for model in models:
    model.compile(
        optimizer=Adam(learning_rate=0.0003, beta_1=0.9),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    model.set_weights(globalweights)

In [ ]:
# Training parameters
epochs = 10  # Increased local epochs
batch_size = 16
set_rounds = 12
RoundsUp = 0
doYouWantToRunMore = "yes"

while doYouWantToRunMore == "yes":
    for i, model in enumerate(models):
        x_train_local, y_train_local = local_datasets[i]
        print(f"Training local model {i+1}...")

        early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-6)

        model.fit(
            data_gen.flow(x_train_local, y_train_local, batch_size=batch_size),
            epochs=epochs,
            callbacks=[early_stopping, reduce_lr]
        )
    
    averaged_weights = fedavg(*models)
    global_model.set_weights(averaged_weights)
    for model in models:
        model.set_weights(averaged_weights)
    
    loss, accuracy = global_model.evaluate(test_set, test_labels, batch_size=8)
    print(f'Round {RoundsUp + 1}: Loss = {loss}, Accuracy = {accuracy}')
    RoundsUp += 1